In [103]:
import sqlite3
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

In [104]:
import pandas as pd
df = pd.read_csv("dataframe_updates.csv", index_col=0)
df.sample(10)

x     id  scooters_on_parking     timezone  \
50433  2022-06-01 14:00:00.000  50088                   12       Moscow   
74554  2022-06-01 20:30:00.000  65175                    8       Moscow   
9573   2022-06-01 02:30:00.000  51171                    0  Vladivostor   
2644   2022-06-01 00:30:00.000  53925                    0  Vladivostor   
83787  2022-06-01 23:00:00.000  72805                    0  Vladivostor   
67461  2022-06-01 18:30:00.000  65430                    8       Moscow   
52757  2022-06-01 14:30:00.000  57414                    0       Moscow   
60018  2022-06-01 16:30:00.000  58941                    0       Moscow   
12228  2022-06-01 03:00:00.000  65513                    0       Moscow   
70025  2022-06-01 19:30:00.000  50020                    0       Moscow   

       starts  
50433       2  
74554       3  
9573        5  
2644        6  
83787       5  
67461       3  
52757       5  
60018       5  
12228       3  
70025       6

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87419 entries, 0 to 87418
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   x                    87419 non-null  object
 1   id                   87419 non-null  int64 
 2   scooters_on_parking  87419 non-null  int64 
 3   timezone             87419 non-null  object
 4   starts               87419 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 4.0+ MB


таблица с агрегированными по 30 минут данными – dataframe_updates.csv  
Поля в таблице содержат следующие данные:  
● x - время по UTC,  
● id - фиксированная точка в пространстве,  
● scooters_on_parking – количество самокатов в данной точке на начало указанного временного промежутка,  
● timezone – опорная точка по которой можно ориентироваться на таймзону.  
● starts - количество стартов с парковки  

In [106]:
df['x'] = pd.to_datetime(df['x'])

In [107]:
df = df.rename(columns={'x': 'time'})

In [108]:
df.head()

time     id  scooters_on_parking timezone  starts
0 2022-06-01  49383                    8   Moscow       3
1 2022-06-01  49384                    0   Moscow       5
2 2022-06-01  49386                    0   Moscow       4
3 2022-06-01  49387                    0   Moscow       1
4 2022-06-01  49388                    8   Moscow       9

In [109]:
df.time.describe()

count                            87419
mean     2022-06-01 12:00:01.482515200
min                2022-06-01 00:00:00
25%                2022-06-01 06:00:00
50%                2022-06-01 12:00:00
75%                2022-06-01 18:00:00
max                2022-06-02 00:00:00
Name: time, dtype: object

In [110]:
df.time.unique()

<DatetimeArray>
['2022-06-01 00:00:00', '2022-06-01 00:30:00', '2022-06-01 01:00:00',
 '2022-06-01 01:30:00', '2022-06-01 02:00:00', '2022-06-01 02:30:00',
 '2022-06-01 03:00:00', '2022-06-01 03:30:00', '2022-06-01 04:00:00',
 '2022-06-01 04:30:00', '2022-06-01 05:00:00', '2022-06-01 05:30:00',
 '2022-06-01 06:00:00', '2022-06-01 06:30:00', '2022-06-01 07:00:00',
 '2022-06-01 07:30:00', '2022-06-01 08:00:00', '2022-06-01 08:30:00',
 '2022-06-01 09:00:00', '2022-06-01 09:30:00', '2022-06-01 10:00:00',
 '2022-06-01 10:30:00', '2022-06-01 11:00:00', '2022-06-01 11:30:00',
 '2022-06-01 12:00:00', '2022-06-01 12:30:00', '2022-06-01 13:00:00',
 '2022-06-01 13:30:00', '2022-06-01 14:00:00', '2022-06-01 14:30:00',
 '2022-06-01 15:00:00', '2022-06-01 15:30:00', '2022-06-01 16:00:00',
 '2022-06-01 16:30:00', '2022-06-01 17:00:00', '2022-06-01 17:30:00',
 '2022-06-01 18:00:00', '2022-06-01 18:30:00', '2022-06-01 19:00:00',
 '2022-06-01 19:30:00', '2022-06-01 20:00:00', '2022-06-01 20:30:00',
 '20

In [111]:
df.to_sql("scooter", conn, if_exists="replace")

87419

1.Сгруппировать записи стартов по 3 часа и по id пространственных точек, чтобы
получить суммарное количество стартов для данного временного промежутка в
этой точке

In [112]:
cur.execute("""
    CREATE TEMPORARY TABLE temp_table AS
    SELECT
        strftime('%Y-%m-%d %H:00:00', datetime((strftime('%s', time) / (3 * 3600)) * (3 * 3600), 'unixepoch')) AS time_slot,
        id,
        SUM(starts) AS total_starts
    FROM scooter
    GROUP BY time_slot, id
""")

cur.execute("SELECT * FROM temp_table")
rows = cur.fetchall()

for i, row in enumerate(rows):
    if i < 5:
        print(row)

('2022-06-01 00:00:00', 49383, 31)
('2022-06-01 00:00:00', 49384, 39)
('2022-06-01 00:00:00', 49386, 24)
('2022-06-01 00:00:00', 49387, 27)
('2022-06-01 00:00:00', 49388, 33)


проверка

In [113]:
df = df.set_index('time')
df.head()

id  scooters_on_parking timezone  starts
time                                                   
2022-06-01  49383                    8   Moscow       3
2022-06-01  49384                    0   Moscow       5
2022-06-01  49386                    0   Moscow       4
2022-06-01  49387                    0   Moscow       1
2022-06-01  49388                    8   Moscow       9

In [114]:
df_resampled = df.groupby(['id', pd.Grouper(freq='3H')]).sum()
df_result = df_resampled['starts'].reset_index().sort_values(by=['time', 'id'])
df_result.head()

id       time  starts
0   49383 2022-06-01      31
9   49384 2022-06-01      39
18  49386 2022-06-01      24
27  49387 2022-06-01      27
36  49388 2022-06-01      33

2.Помимо поля с общим количеством стартов, добавить значение количества
самокатов за последний временной промежуток на основание scooters_on_
parking (последние полчаса до заданноговременного промежутка)

In [115]:
cur.execute("""
    CREATE INDEX idx_updates_time ON scooter(time)
""")

cur.execute("""
    CREATE INDEX idx_updates_id ON scooter(id)
""")

cur.execute("""
    CREATE INDEX idx_temp_table_time_slot ON temp_table(time_slot)
""")

cur.execute("""
   CREATE INDEX idx_temp_table_id ON temp_table(id)
""")

In [116]:
cur.execute("""
    ALTER TABLE temp_table ADD COLUMN last_scooters INTEGER
""")

cur.execute("SELECT * FROM temp_table")
rows = cur.fetchall()

for i, row in enumerate(rows):
    if i < 5:
        print(row)

('2022-06-01 00:00:00', 49383, 31, None)
('2022-06-01 00:00:00', 49384, 39, None)
('2022-06-01 00:00:00', 49386, 24, None)
('2022-06-01 00:00:00', 49387, 27, None)
('2022-06-01 00:00:00', 49388, 33, None)


In [117]:
cur.execute("""
    UPDATE temp_table
        SET last_scooters = (
            SELECT scooters_on_parking
            FROM scooter
            WHERE
                datetime(scooter.time) = datetime(temp_table.time_slot, '-30 minutes')
                AND scooter.id = temp_table.id
        )
""")
cur.execute("SELECT * FROM temp_table")
rows = cur.fetchall()

for i, row in enumerate(rows):
    if i < 5:
        print(row)

('2022-06-01 00:00:00', 49383, 31, None)
('2022-06-01 00:00:00', 49384, 39, None)
('2022-06-01 00:00:00', 49386, 24, None)
('2022-06-01 00:00:00', 49387, 27, None)
('2022-06-01 00:00:00', 49388, 33, None)


In [121]:
df_result['time_minus_30'] = df_result['time'] - pd.Timedelta(minutes=30)
df_result.head()

id       time  starts       time_minus_30
0   49383 2022-06-01      31 2022-05-31 23:30:00
9   49384 2022-06-01      39 2022-05-31 23:30:00
18  49386 2022-06-01      24 2022-05-31 23:30:00
27  49387 2022-06-01      27 2022-05-31 23:30:00
36  49388 2022-06-01      33 2022-05-31 23:30:00

In [123]:
merged_df = df_result.merge(df, left_on=['id', 'time_minus_30'], right_on=['id', 'time'], how='left', suffixes=('', '_y'))
merged_df

id       time  starts       time_minus_30  scooters_on_parking  \
0      49383 2022-06-01      31 2022-05-31 23:30:00                  NaN   
1      49384 2022-06-01      39 2022-05-31 23:30:00                  NaN   
2      49386 2022-06-01      24 2022-05-31 23:30:00                  NaN   
3      49387 2022-06-01      27 2022-05-31 23:30:00                  NaN   
4      49388 2022-06-01      33 2022-05-31 23:30:00                  NaN   
...      ...        ...     ...                 ...                  ...   
16054  83538 2022-06-02       5 2022-06-01 23:30:00                 12.0   
16055  83539 2022-06-02       3 2022-06-01 23:30:00                  0.0   
16056  83561 2022-06-02       5 2022-06-01 23:30:00                  NaN   
16057  83563 2022-06-02       9 2022-06-01 23:30:00                  NaN   
16058  83735 2022-06-02       2 2022-06-01 23:30:00                  NaN   

          timezone  starts_y  
0              NaN       NaN  
1              NaN       NaN  
2              NaN       NaN  
3              NaN       NaN  
4              NaN       NaN  
...            ...       ...  
16054  Vladivostor       5.0  
16055  Vladivostor       3.0  
16056          NaN       NaN  
16057          NaN       NaN  
16058          NaN       NaN  

[16059 rows x 7 columns]

In [125]:
cur.execute("SELECT * FROM temp_table WHERE id = 83538")
rows = cur.fetchall()

for i, row in enumerate(rows):
    if i < 20:
        print(row)

('2022-06-01 00:00:00', 83538, 31, None)
('2022-06-01 03:00:00', 83538, 29, 24)
('2022-06-01 06:00:00', 83538, 30, 28)
('2022-06-01 09:00:00', 83538, 24, 0)
('2022-06-01 12:00:00', 83538, 27, 0)
('2022-06-01 15:00:00', 83538, 32, 4)
('2022-06-01 18:00:00', 83538, 42, 4)
('2022-06-01 21:00:00', 83538, 29, 4)
('2022-06-02 00:00:00', 83538, 5, 12)


3. Привести время к нужной таймзоне, сейчас оно в формате UTC

In [126]:
df.timezone.unique()

array(['Moscow', 'Vladivostor'], dtype=object)

In [128]:
cur.execute("""
    UPDATE temp_table
    SET time_slot = datetime(
        time_slot, 
        '+' || (
            SELECT 
                CASE 
                    WHEN timezone = 'Moscow' THEN '3'
                    WHEN timezone = 'Vladivostok' THEN '10'
                    ELSE '0' -- Если не найдено соответствие, не изменять время
                END || ' hours'
            FROM scooter
            WHERE scooter.id = temp_table.id
            LIMIT 1
        )
    )
""")
cur.execute("SELECT * FROM temp_table")
rows = cur.fetchall()

for i, row in enumerate(rows):
    if i < 5:
        print(row)

('2022-06-01 03:00:00', 49383, 31, None)
('2022-06-01 03:00:00', 49384, 39, None)
('2022-06-01 03:00:00', 49386, 24, None)
('2022-06-01 03:00:00', 49387, 27, None)
('2022-06-01 03:00:00', 49388, 33, None)


4. Предложить решение, как инкрементально обновлять эти данные, если исходная таблица будет обновляться каждые 3 часа

Не понятно какая таблица будет обновляться потому что я тут моделировал sql. Будет обновляться csv? Странновато звучит, ну наверно sql таблица
Тогда просто в запросе выбираем только записи из основной таблицы, которые были добавлены или изменены после последнего обновления агрегированной таблицы